# 9. User Interface Development with Streamlit

In [ ]:
# Install necessary packages
!pip install fastapi uvicorn pyngrok nest-asyncio streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
# Standard library imports
import os

# Third-party library imports
import requests
import streamlit as st
from threading import Thread
from pyngrok import ngrok

# Colab-specific imports
from google.colab import userdata

## Employee Attrition Prediction Web App with Streamlit and FastAPI

This code creates an interactive **Streamlit** app that allows users to input employee details and get predictions on employee attrition (whether the employee will leave or stay). The app communicates with a **FastAPI** backend model, which performs the prediction.

### Code Overview

1. **FastAPI Integration**:
   - The FastAPI model URL is read from the `fastapi_url.txt` file.
   - The Streamlit app sends a POST request with employee details to the FastAPI endpoint for prediction.

2. **User Input Fields**:
   - **Education**: User selects the employee's education level (`Bachelors`, `Masters`, `PHD`).
   - **Joining Year**: User enters the employee's joining year (range between 2000–2025).
   - **City**: User selects the employee's city (`Bangalore`, `Pune`, `New Delhi`).
   - **Payment Tier**: User selects the payment tier of the employee (`1`, `2`, `3`).
   - **Age**: User enters the age of the employee (between 18 and 65).
   - **Gender**: User selects the gender of the employee (`Male`, `Female`).
   - **Ever Benched**: User selects whether the employee has ever been benched (binary: `0` for No, `1` for Yes).
   - **Experience in Current Domain**: User enters the number of years of experience in the current domain.

3. **Prediction Request**:
   - Once the user provides the input and clicks **Predict**, the app sends the data to the FastAPI backend.
   - The model predicts whether the employee will `Leave` or `Stay`, and the result is displayed.

4. **Error Handling**:
   - The app handles potential connection issues by displaying an error message if the API is not reachable.

### Code Execution Flow

1. **API URL Retrieval**:
   The FastAPI URL is fetched from a file (`fastapi_url.txt`), which contains the prediction endpoint.

2. **Streamlit UI**:
   - Various input fields are provided for the user to enter details about the employee.
   - The `Predict` button sends the input data to the FastAPI service via a POST request.

3. **Prediction**:
   - The FastAPI backend processes the input data and returns a prediction, which is displayed as either **Stay** or **Leave** based on the model’s output.
   - If an error occurs during the request (e.g., connectivity issues), a user-friendly error message is shown.


### Key Points
- **User Input Fields**: Employee details such as Education, Age, Experience, etc., are input by the user.
- **Prediction API**: The user input is sent to the FastAPI model, which performs the prediction.
- **Streamlit Interface**: The results are displayed on the Streamlit web page, making it easy for users to interact and get predictions.





In [ ]:
%%writefile app.py
# Import necessary libraries
import requests  # For making HTTP requests
import streamlit as st  # For building the Streamlit web app

# Load FastAPI public URL dynamically from the file
with open("fastapi_url.txt", "r") as f:
    api_url = f.read().strip() + "/predict"

# Streamlit app title and description
st.title("Employee Attrition Prediction")
st.write("Use this app to predict whether an employee will leave or stay.")

# Input fields for user data
# Education level
education = st.selectbox("Education", options=["Bachelors", "Masters", "PHD"])

# Joining year with a range of valid years
joining_year = st.number_input(
    "Joining Year", min_value=2000, max_value=2025, step=1, value=2015
)

# City options
city = st.selectbox("City", options=["Bangalore", "Pune", "New Delhi"])

# Payment tier selection
payment_tier = st.selectbox("Payment Tier", options=[1, 2, 3])

# Employee age input with range validation
age = st.number_input("Age", min_value=18, max_value=65, step=1, value=30)

# Gender selection
gender = st.selectbox("Gender", options=["Male", "Female"])

# Ever benched (binary: 0 for "No", 1 for "Yes")
ever_benched = st.selectbox(
    "Ever Benched", options=[0, 1], format_func=lambda x: "No" if x == 0 else "Yes"
)

# Experience in the current domain
experience_in_current_domain = st.number_input(
    "Experience in Current Domain (years)", min_value=0, max_value=80, step=1, value=5
)

# Button to trigger prediction
if st.button("Predict"):
    # Prepare input data for the API
    input_data = {
        "Education": education,
        "JoiningYear": joining_year,
        "City": city,
        "PaymentTier": payment_tier,
        "Age": age,
        "Gender": gender,
        "EverBenched": ever_benched,
        "ExperienceInCurrentDomain": experience_in_current_domain,
    }

    try:
        # Send POST request to the FastAPI prediction endpoint
        response = requests.post(api_url, json=input_data)
        response.raise_for_status()  # Raise exception for HTTP errors

        # Parse response and display prediction
        response_data = response.json()
        prediction = "Stay" if response_data["prediction"] == 0 else "Leave"
        st.success(f"Prediction: {prediction}")
    except requests.exceptions.RequestException as e:
        # Handle errors and display an appropriate message
        st.error(f"Error connecting to the prediction API: {e}")

Writing app.py


## Start Streamlit Server and Expose via Ngrok

This code snippet runs the **Streamlit** app in a separate thread and exposes it to the internet using **Ngrok**. It enables remote access to the web interface of the Employee Attrition Prediction app, which is running on a local server.

### Code Overview

1. **Start Streamlit Server**:
   - The `streamlit_thread` is a background thread that starts the Streamlit app (`app.py`) on port `8501` using the command:
     ```bash
     streamlit run app.py --server.port 8501
     ```

2. **Expose via Ngrok**:
   - The **Ngrok** service is used to create a public URL for the local Streamlit app.
   - The public URL is printed and can be shared with others to access the app remotely.


### Code Execution Flow

1. **Streamlit App**:
   - The Streamlit app (`app.py`) is launched in a separate thread, which allows the notebook to continue executing other tasks without blocking.

2. **Ngrok Tunnel**:
   - Ngrok is used to expose the locally running Streamlit app to the internet.
   - The URL provided by Ngrok can be shared and accessed in a browser to interact with the app.

3. **Public URL**:
   - Once the Streamlit server is running, the public URL generated by Ngrok is displayed in the output. This URL can be used to access the app from any browser.

### Key Points
- **Streamlit in Background**: The Streamlit server is started in a background thread to prevent blocking the notebook execution.
- **Ngrok Exposure**: Ngrok creates a secure tunnel, exposing the local Streamlit app to the internet with a public URL.
- **Accessing the App**: After running this code, the user can visit the public URL to interact with the app.



In [ ]:
# Start the Streamlit server in a separate thread
streamlit_thread = Thread(
    target=lambda: os.system("streamlit run app.py --server.port 8501"), daemon=True
)
streamlit_thread.start()
auth_token = userdata.get("Ngrok")
ngrok.set_auth_token(auth_token)
# Expose the Streamlit app through ngrok
# Ensure fastapi_url.txt file exists in the Colab environment- Get it from Notebook-2
streamlit_url = ngrok.connect(8501)
print(f"Streamlit public URL: {streamlit_url}")

Streamlit public URL: NgrokTunnel: "https://7629-34-16-134-239.ngrok-free.app" -> "http://localhost:8501"
